In [ ]:
from traitlets import Unicode, Bool, validate, TraitError, observe
from ipywidgets import DOMWidget, register
import time
import threading

In [ ]:
%%javascript

require.undef('email_widget');

// sleep time expects milliseconds
function sleep (time) {
    return new Promise((resolve) => setTimeout(resolve, time));
}

define('email_widget', ["@jupyter-widgets/base"], function(widgets) {
    var EmailView = widgets.DOMWidgetView.extend({
        render: function() {
            this.email_input = document.createElement('input');
            this.email_input.type = 'email';
            this.email_input.disabled = true;
            this.email_input.value = this.model.get('jspy_channel');
            this.el.appendChild(this.email_input);
            
            // Python -> JavaScript update
            this.model.on('change:pyjs_channel', this.on_pyjs_message, this);
            
            // JavaScript -> Python update
            sleep(1000).then(() => {
                this.model.set('jspy_channel', "Hello from JS: A");
                this.touch();
            }).then(() => sleep(1000)).then(() => {
                this.model.set('jspy_channel', "Hello from JS: B");
                this.touch();
            }).then(() => sleep(1000)).then(() => {
                this.model.set('jspy_channel', "Hello from JS: C");
                this.touch();
            });
        },
        
        on_pyjs_message: function() {
            this.email_input.value = this.model.get('pyjs_channel');
        },
        
    });
    
    return {
        EmailView: EmailView
    }
});

In [ ]:
def py2js_send_five_messages(widget):
    def func():
        for i in range(3):
            widget.pyjs_channel = f"Hello from Python: {i+1}"
            time.sleep(1)
    return func

@register
class WebVisualizer(DOMWidget):
    _view_name = Unicode('EmailView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes
    pyjs_channel = Unicode("Empty pyjs_channel.", help="Python->JS messaging channel.").tag(sync=True)
    jspy_channel = Unicode("Empty jspy_channel.", help="JS->Python messaging channel.").tag(sync=True)

    @observe('jspy_channel')
    def on_jspy_message(self, change):
        print(f"jspy message: {change['new']}")

In [ ]:
visualizer = WebVisualizer()
visualizer

In [ ]:
thread = threading.Thread(target=py2js_send_five_messages(visualizer))
thread.start()